In [2]:
import os
import time 

import random
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import numpy as np
import cv2

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [3]:
# Random seed for reproducibility
seed = 55

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
def add_noise(img):
    '''Add random noise to an image'''
    VARIABILITY = 70
    deviation = VARIABILITY*random.random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img

def change_color(image):
    image = np.array(image)
    hsv_image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    return hsv_image

def preprocessing(image):
    CP1 = 0.3
    CP2 = 0.6
    n = random.random()
    if n<=CP1: 
        return change_color(image)
    elif n>CP1 and n<=CP2:
        return add_noise(image)
    return image

In [5]:
training_dir = "../input/training/training"
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_gen = ImageDataGenerator(rescale=1/255, validation_split=0.2)
aug_train_data_gen = ImageDataGenerator(rotation_range=30, 
                                        height_shift_range=50, 
                                        width_shift_range=50, 
                                        zoom_range=0.3, 
                                        horizontal_flip=True, 
                                        vertical_flip=True,  
                                        fill_mode='reflect',
                                        preprocessing_function=preprocessing,
                                        rescale=1/255, 
                                        validation_split=0.2)
train_gen = aug_train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=64,
                                               shuffle=True,
                                               seed=seed,
                                               subset="training")

valid_gen = data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=1,
                                               shuffle=True,
                                               seed=seed,
                                               subset="validation")

In [6]:
input_shape = (256, 256, 3)
epochs = 50

In [7]:
# Download and plot the VGG16 model
supernet = tfk.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3)
)

In [9]:
for layer in supernet.layers:
    layer.trainable = False

In [10]:
inputs = tfk.Input(shape=(256,256,3))
x = supernet(inputs)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dense(
    512, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.2, seed=seed)(x)
x = tfkl.Dense(
    512, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.2, seed=seed)(x)
outputs = tfkl.Dense(
    14, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

In [11]:
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_augmentation_experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [12]:
# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')
tl_model.summary()

In [13]:
callbacks = create_folders_and_callbacks(model_name='CNN')
# Train the model
tl_history = tl_model.fit(
    train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = callbacks
).history

In [ ]:
# Save the model
tl_model.save('TransferLearningModel')

In [ ]:
# Load the model
ft_model = tfk.models.load_model('TransferLearningModel')
ft_model.summary()

In [ ]:
# Set all Inception layers to True
ft_model.get_layer('inception_v3').trainable = True

In [ ]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')
ft_model.summary()

In [ ]:
callbacks = create_folders_and_callbacks(model_name='CNN')
# Train the model
ft_history = ft_model.fit(
    train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    class_weight = {0: 5693/988, 1: 5693/467, 2: 5693/583, 3: 5693/1206, 4: 5693/1458, 
                    5: 5693/1748, 6: 5693/977, 7: 5693/765, 8: 5693/716, 9: 5693/264, 
                    10: 5693/1616, 11: 5693/574, 12: 5693/673, 13: 1.},
    callbacks = callbacks
).history

In [ ]:
# Save the best model
ft_model.save('FineTuning_Model')